<a href="https://colab.research.google.com/github/jinyingtld/python/blob/main/AI6126_tutorial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[SRCNN论文详解（Image Super-resolution Using Deep Convolutional Networks）](https://www.jianshu.com/p/d73d206943eb)
## 1.简介:
超分辨率(super resolution)的任务目标时将输入的低分辨率的图像转换为高分辨率的图像, 与图像去噪,图像去模糊等一脉相承. 个人认为：超分辨率关注的是从小尺寸到大尺寸图像如何填充新的像素；图像去噪则是关注在图像尺寸不变的情况下，将被“污染”的像素替换为正确的像素。

SRCNN是首个使用CNN结构(即基于深度学习)的端倒短的超分辨率算法,这个相当于什么? 就像faster r-cnn在目标检测的低位一样, 将整个算法流程用深度学习的方法实现了, 并且效果比传统多模块集成的方法好. SRCNN流程如下:首先, 输入预处理. 对输入的低分辨率LR(Low Resolution)图像使用的bicubic算法进行放大, 放大为目标尺寸. 接下来算法的目标就是将输入的比较模糊的LR图像, 经过卷积网络的处理, 得到超分辨率SR(Super Resolution)的图像, 使它尽可能与原图的高分辨率HR图像相似.

## 2. 模型及训练
[SRCNN 网络结构](https://img-blog.csdnimg.cn/20200420101224696.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80NjU1OTI3MQ==,size_16,color_FFFFFF,t_70)

SRCNN的网络结构比较简单, 这个卷积网络包括三个卷积层, 甚至没有池化和全连接层:
1. 对低分辨率的图像进行卷积操作, 生成n1维的feature maps
2. 对n1维的feature map进行卷积操作生成n2维feature maps
3. 对n2维的feature maps进行卷积生成超分辨率的图像.

分别对应三个处理流程:
1. **提取图像特征:** 从低分辨率突袭那个中提取多个patch图像块, 每个块被卷积操作表示为多维的向量(维数等于filter的数量), 所有的特征向量组成特征矩阵(feature maps)
2. **非线性映射:** 将n1维特征矩阵,通过卷积操作实现非线性映射, 变成另一n2维特征矩阵. 
3. **重构图像:** 等于是个反卷积的过程, 将n2的特征矩阵还原为超分辨率图像. 

模型训练的优化参数是三层卷积层对应的卷积和(w)和bias(b), 参数P={W_1,W_2,W_3,b_1,b_2,b_3}. 训练的目标损失是最小化超分辨率图像F(Y;P)与原高分辨率图像X基于像素的均方差MSE,定义如下:
[Loss](https://upload-images.jianshu.io/upload_images/3940902-49808ffab773f063?imageMogr2/auto-orient/strip|imageView2/2/w/344/format/webp) [网址](https://www.jianshu.com/p/d73d206943eb)

其中n是训练样本数量, 即每次训练的样本数量. 接下来无非就是随机梯度下降法反向传播, 网络训练得到最终的参数P使损失L最小化, 参数跟新公式如下:
[参数更新](https://upload-images.jianshu.io/upload_images/3940902-f183b6931966c8f5.png?imageMogr2/auto-orient/strip|imageView2/2/w/571/format/webp) [网址](https://www.jianshu.com/p/d73d206943eb)

## 3. 实验
### 3.1 对比卷积核大小(filter size), 卷积核数量(filter numbers)
**对复原效果的影响的实验**
**结论:** 卷积核数量越多,即特征向量维数越高, 实验效果越好, 但是会影响算法速度, 故需要综合考虑; 另外三层卷积层的卷积和大小越大, 实验效果也会略微更好, 同样会影响算法速度.

### 3.2 对比网络层数(layer numbers) 对复原效果的影响的实验
**结论:** 并非网络越深，效果越好，结果恰恰相反。作者也给出了解释：因为SRCNN没有池化层和全连接层，导致网络对初始参数和学习率非常敏感，结果即网络训练的时候非常难以收敛，即使收敛了也可能停在了坏的局部最小值（bad local minimum）处，并且即使训练了足够的时间，学习到的filter参数的分散度也不够好。

### 3.3 与最前沿的其他超分算法对比速度与性能的实验

数据集选择的是ImageNet上的BSD200、Set14、Set5，对比对象是Bicubic、SC、NE+LLE、KK、ANR、A+与SRCNN。结果如下：
[Set5数据集上实验结果](https://upload-images.jianshu.io/upload_images/3940902-20623e7818ce2801.png?imageMogr2/auto-orient/strip|imageView2/2/w/1200/format/webp)

[Set14数据集上实验结果](https://upload-images.jianshu.io/upload_images/3940902-7316c3d074fe82b2.png?imageMogr2/auto-orient/strip|imageView2/2/w/1200/format/webp)

[BSD200数据集上实验结果](https://upload-images.jianshu.io/upload_images/3940902-7798fd65bc2659cd.png?imageMogr2/auto-orient/strip|imageView2/2/w/1200/format/webp)

[对比算法速度](https://upload-images.jianshu.io/upload_images/3940902-0a3e48deaca24cc2.png?imageMogr2/auto-orient/strip|imageView2/2/w/583/format/webp)

**结论:**
对于复原质量，SRCNN在大部分指标上都表现最好。超分辨率问题（SR）用PSNR作为重建的指标，但是高PSNR并不意味着重建结果好。因为SR是病态问题，这就意味着可能解有无穷多个。以MSE（等价于PSNR）为loss函数得到的解往往是这些可能解的平均，而不是最优解。这就导致以MSE为loss的SR算法得到的结果有些区域看起来过于模糊，缺乏应有的细节。这也是为什么出现了数值评估低，但肉眼看着效果更好的现象。故作者选择了很多项指标来评价复原图像的质量；

对于复原速度，前面提到卷积核大小会影响速度，故作者对比了三种不同卷积核大小的SRCNN与其他算法的速度，总的来说SRCNN的复原速度属于前列水平。

### 3.4  通道对复原效果影响的实验
**结论:** RGB通道联合训练效果最好；YCbCr通道下，Cb、Cr通道对性能提升基本无帮助，只基于Y通道的训练效果更好。

## 4. 小结
SRCNN提出轻量的端到端网络SRCNN来解决超分问题，的确在当时取得了比传统方法性能更强、速度更快的效果，另外作者将基于SC(稀疏编码)的超分方法理解为卷积神经网络的一种形式，都是非常值得阅读的亮点。
